In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib qt

In [40]:
mydict = [{'a': 1, 'b': 2, 'c': 3, 'd': 4},
          {'a': 100, 'b': 200, 'c': 300, 'd': 400},
          {'a': 2000, 'b': 220, 'c': 4000, 'd': 24 }]
df = pd.DataFrame(mydict)
df

,a,b,c,d
0,1,2,3,4
1,100,200,300,400
2,2000,220,4000,24


In [41]:
n = df.index[df['b'] == 200]
n[0]

1

In [43]:
df.sort_values('d', ascending=True)

,a,b,c,d
0,1,2,3,4
2,2000,220,4000,24
1,100,200,300,400
